In [20]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [21]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [22]:
sess = tf.InteractiveSession()

In [23]:
x = tf.placeholder(tf.float32,shape=[None,784])
y = tf.placeholder(tf.float32,shape=[None,10])

In [24]:
x_image = tf.reshape(x,[-1,28,28,1],name="x_image")

In [25]:
def weight_variable(shape):
    intial= tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(intial)


In [26]:
def bias_variable(shape):
    intial = tf.constant(0.1,shape=shape)
    return tf.Variable(intial)

In [27]:
def conv_2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding = "SAME")

In [28]:
def max_pooling_2d(x):
    return tf.nn.max_pool(x,strides=[1,2,2,1],ksize=[1,2,2,1],padding = "SAME")


In [29]:
W_conv_1 = weight_variable([5,5,1,32])
bias_conv_1 = bias_variable([32])

hid_conv_1 = tf.nn.relu(conv_2d(x_image,W_conv_1)+bias_conv_1)
hid_pool_1 = max_pooling_2d(hid_conv_1)

In [30]:
W_conv_2 = weight_variable([5,5,32,64])
bias_conv_2 = bias_variable([64])

hid_conv_2 = tf.nn.relu(conv_2d(hid_pool_1,W_conv_2)+bias_conv_2)
hid_pool_2 = max_pooling_2d(hid_conv_2)

In [31]:
W_full_1 = weight_variable([7*7*64,1024])
bias_full_1 = bias_variable([1024])

h_pool_2flat = tf.reshape(hid_pool_2,[-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool_2flat,W_full_1)+bias_full_1)
keep_probability = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_probability)

In [32]:
W_fc2 = weight_variable([1024,10])
bias_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop,W_fc2)+bias_fc2

In [34]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_conv))

optimizer = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [35]:
sess.run(tf.global_variables_initializer())

In [36]:
import time

num_steps = 1000
display_every = 100

start_time = time.time()
end_time = time.time()

In [37]:
for i in range(num_steps):
    batch = mnist.train.next_batch(50)
    optimizer.run(feed_dict={x: batch[0], y: batch[1], keep_probability: 0.5})
    
    if i%display_every == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y: batch[1], keep_probability: 1.0})
        end_time = time.time()
        print("step {0}, elapsed time {1:.2f} seconds, training accuracy {2:.3f}%".format(i, end_time-start_time, train_accuracy*100.0))
  
end_time = time.time()

step 0, elapsed time 9.46 seconds, training accuracy 14.000%
step 100, elapsed time 32.37 seconds, training accuracy 80.000%
step 200, elapsed time 58.32 seconds, training accuracy 94.000%
step 300, elapsed time 82.64 seconds, training accuracy 92.000%
step 400, elapsed time 106.95 seconds, training accuracy 90.000%
step 500, elapsed time 131.32 seconds, training accuracy 92.000%
step 600, elapsed time 155.65 seconds, training accuracy 92.000%
step 700, elapsed time 180.10 seconds, training accuracy 92.000%
step 800, elapsed time 204.42 seconds, training accuracy 96.000%
step 900, elapsed time 228.56 seconds, training accuracy 98.000%


In [38]:
print("Total training time for {0} batches: {1:.2f} seconds".format(i+1, end_time-start_time))

#     Accuracy on test data
print("Test accuracy {0:.3f}%".format(accuracy.eval(feed_dict={
    x: mnist.test.images, y: mnist.test.labels, keep_probability: 1.0})*100.0))

sess.close()

Total training time for 1000 batches: 252.59 seconds
Test accuracy 96.340%
